In [ ]:
!pip install emoji
!pip install ekphrasis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.8/83.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 2.2 MB/s eta 0:00:00


In [ ]:
pip install plotly==4.5.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 94.7 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.24.1
    Uninstalling plotly-5.24.1:


In [ ]:
!pip install transformers==4.2.1

In [ ]:
!pip install ekphrasis

In [ ]:
!pip install emoji

In [ ]:
import numpy as np
import pandas as pd
import string
from nltk.corpus import stopwords
import re
import os

from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

In [ ]:
text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'url', 'date', 'number'],
    # terms that will be annotated
    annotate={"hashtag", "allcaps", "elongated", "repeated",
        'emphasis', 'censored'},
    fix_html=True,  # fix HTML tokens

    # corpus from which the word statistics are going to be used
    # for word segmentation
    segmenter="twitter",

    # corpus from which the word statistics are going to be used
    # for spell correction
    corrector="twitter",

    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=True,  # spell correction for elongated words

    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,

    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)

In [ ]:
def print_text(texts,i,j):
    for u in range(i,j):
        print(texts[u])
        print()

In [ ]:
df = pd.read_csv('/content/train.csv',delimiter=',',encoding='utf-8')  # train set
print(list(df.columns.values)) #file header

In [ ]:
text_array = df["text"]
labels = df["label_sexist"]
original = text_array

In [ ]:
from collections import Counter

In [ ]:
df_validate = pd.read_csv('/content/validate.csv',delimiter=',',encoding='utf-8') # validation set

In [ ]:
df_test = pd.read_csv('/content/test.csv',delimiter=',',encoding='utf-8') # test set

In [ ]:
#check if need to change
text_array_val = df_validate["text"]
labels_val = df_validate["label_sexist"]

In [ ]:
len(labels_val)

In [ ]:
test_set_text = df_test["text"]
labels_test_set = df_test["label_sexist"]

In [ ]:
#removing website names
def remove_website(text):
    return " ".join([word if re.search("r'https?://\S+|www\.\S+'|((?i).com$|.co|.net)",word)==None else "" for word in text.split(" ") ])

# Training set
text_array = text_array.apply(lambda text: remove_website(text))
print_text(text_array,0,10)

print("**************************************************************************")

# Validation set
text_array_val = text_array_val.apply(lambda text: remove_website(text))
print_text(text_array_val,0,10)

print("**************************************************************************")

# Test set
test_set_text = test_set_text.apply(lambda text: remove_website(text))
print_text(test_set_text,0,10)

In [ ]:
# Functions for chat word conversion
f = open("/content/slang.txt", "r")
chat_words_str = f.read()
chat_words_map_dict = {}
chat_words_list = []

for line in chat_words_str.split("\n"):
    if line != "":
        cw = line.split("=")[0]
        cw_expanded = line.split("=")[1]
        chat_words_list.append(cw)
        chat_words_map_dict[cw] = cw_expanded
chat_words_list = set(chat_words_list)

def chat_words_conversion(text):
    new_text = []
    for w in text.split():
        if w.upper() in chat_words_list:
            new_text.append(chat_words_map_dict[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text)

In [ ]:
# Chat word conversion
# Training set
text_array = text_array.apply(lambda text: chat_words_conversion(text))
print_text(text_array,0,10)
print_text(original,0,10)

print("********************************************************************************")

# Validation set
text_array_val = text_array_val.apply(lambda text: chat_words_conversion(text))
print_text(text_array_val,0,10)

print("********************************************************************************")

# Test set
test_set_text = test_set_text.apply(lambda text: chat_words_conversion(text))
print_text(test_set_text,0,10)

In [ ]:
#check code in other copy
import re

# Define a dictionary of emoticons and their corresponding meanings
EMOTICONS = {
    ':)': 'smile',
    ':-)': 'smile',
    ':(': 'sad',
    ':-(': 'sad',
    ':D': 'laugh',
    '<3': 'love'
}

def convert_emoticons(text):
    # Iterate over each emoticon in the dictionary
    for emot, meaning in EMOTICONS.items():
        # Replace the emoticon with its corresponding meaning in the text
        text = re.sub(r'(?i)' + re.escape(emot), meaning, text)
    return text

# Test the emoticon conversion function
text = "Hello :-) :-("
text = convert_emoticons(text)
print(text)


In [ ]:
# Emoticon conversion
# Training set
text_array = text_array.apply(lambda text: convert_emoticons(text))
print_text(text_array,0,10)

print("**********************************************************************************")

# Validation set
text_array_val = text_array_val.apply(lambda text: convert_emoticons(text))
print_text(text_array_val,0,10)

print("**********************************************************************************")

# Test set
test_set_text = test_set_text.apply(lambda text: convert_emoticons(text))
print_text(test_set_text,0,10)

In [ ]:
# FUnction for removal of emoji
import emoji

def convert_emojis(text):
    text = emoji.demojize(text, delimiters=(" ", " "))
    text = re.sub("_|-"," ",text)
    return text

# Training set
text_array = text_array.apply(lambda text: convert_emojis(text))
print_text(text_array,0,10)

print("**************************************************************************")

# Validation set
text_array_val = text_array_val.apply(lambda text: convert_emojis(text))
print_text(text_array_val,0,10)

print("**************************************************************************")

# Test set
test_set_text = test_set_text.apply(lambda text: convert_emojis(text))
print_text(test_set_text,0,10)

In [ ]:
# Ekphrasis pipe for text pre-processing
def ekphrasis_pipe(sentence):
    cleaned_sentence = " ".join(text_processor.pre_process_doc(sentence))
    return cleaned_sentence

# Training set
text_array = text_array.apply(lambda text: ekphrasis_pipe(text))
print("Training set completed.......")
#Validation set
text_array_val = text_array_val.apply(lambda text: ekphrasis_pipe(text))
print("Validation set completed.......")
#Test set
test_set_text = test_set_text.apply(lambda text: ekphrasis_pipe(text))
print("Test set completed.......")

In [ ]:
# Removing unnecessary punctuations
PUNCT_TO_REMOVE = "\"$%&'()+,-./;=[\]^_`{|}~"
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

# Training set
text_array = text_array.apply(lambda text: remove_punctuation(text))
print_text(text_array,0,10)

print("********************************************************************")

# Validation set
text_array_val = text_array_val.apply(lambda text: remove_punctuation(text))
print_text(text_array_val,0,10)

print("********************************************************************")

# Test set
test_set_text = test_set_text.apply(lambda text: remove_punctuation(text))
print_text(test_set_text,0,10)

Text pre-processing complete

In [ ]:
# Count of each label in dataset
from collections import Counter

# Printing training set counts for analysis
print("Elements: ",set(labels))
print("Length: ",len(labels))
print(Counter(labels))

print("**************************************************************************")

# Printing validation set counts for analysis
print("Elements: ",set(labels_val))
print("Length: ",len(labels_val))
print(Counter(labels_val))

print("**************************************************************************")

# Printing Test set counts for analysis
print("Elements: ",set(labels_test_set))
print("Length: ",len(labels_test_set))
print(Counter(labels_test_set))

In [ ]:
Y = []
Y_vali = []
Y_test_set = []

# Training set
for i in range(0,len(labels)):
    if(labels[i] == "sexist"):
        Y.append(0)
    if(labels[i] == "not sexist"):
        Y.append(1)

# Validation set
for i in range(0,len(labels_val)):
    if(labels_val[i] == "sexist"):
        Y_vali.append(0)
    if(labels_val[i] == "not sexist"):
        Y_vali.append(1)

# Test set
for i in range(0,len(labels_test_set)):
    if(labels_test_set[i] == "sexist"):
        Y_test_set.append(0)
    if(labels_test_set[i] == "not sexist"):
        Y_test_set.append(1)

In [ ]:
len(labels)

In [ ]:
len(Y)

In [ ]:
len(labels_val)

In [ ]:
len(Y_vali)

In [ ]:
# Verifying train set
X = np.asarray(list(text_array))
Y = np.asarray(list(Y))
print(type(X))
print(type(Y))
print(np.shape(X),np.shape(Y))

# Verifying validation set
X_val = np.asarray(list(text_array_val))
Y_vali = np.asarray(list(Y_vali))
print(type(X_val))
print(type(Y_vali))
print(np.shape(X_val),np.shape(Y_vali))

# Verifying test set
X_test_set = np.asarray(list(test_set_text))
Y_test_set = np.asarray(list(Y_test_set))
print(type(X_test_set))
print(type(Y_test_set))
print(np.shape(X_test_set),np.shape(Y_test_set))

Shuffling training and validation data

In [ ]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [ ]:
# Converting to one hot vectors
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)] #u[Y] helps to index each element of Y index at u. U here is a class array
    return Y

In [ ]:
Y_oh_train = convert_to_one_hot(np.array(Y), C = 2)
Y_oh_val = convert_to_one_hot(np.array(Y_vali), C = 2)
Y_oh_test_set = convert_to_one_hot(np.array(Y_test_set), C = 2)

Model using RoBERTa

In [ ]:
import tensorflow as tf
import os
import numpy as np
import pandas as pd
import string
from nltk.corpus import stopwords
import re
import os
from collections import Counter

In [ ]:
!pip install transformers
from transformers import RobertaTokenizerFast, TFRobertaModel, TFBertModel, BertTokenizerFast, ElectraTokenizerFast, TFElectraModel, AlbertTokenizerFast, TFAlbertModel, XLNetTokenizerFast, TFXLNetModel, MPNetTokenizerFast, TFMPNetModel
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import RepeatVector, Concatenate, Dense, Activation, Dot, BatchNormalization, Dropout

from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

In [ ]:
print(tf.__version__)

In [ ]:
import tensorflow as tf

# Check if a GPU is available
if tf.config.list_physical_devices('GPU'):
    print("GPU is available")
else:
    print("GPU is not available")

# List available GPU devices
print("All devices: ", tf.config.list_logical_devices('GPU'))


In [ ]:
import os
import tensorflow as tf

# Check if TPU address environment variable is set
if 'COLAB_TPU_ADDR' in os.environ:
    tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu=tpu_address)
    tf.config.experimental_connect_to_cluster(resolver)
    tf.tpu.experimental.initialize_tpu_system(resolver)
    print("All devices: ", tf.config.list_logical_devices('TPU'))
else:
    print("TPU not found. Please ensure that TPU is enabled in your runtime settings.")


In [ ]:
import os
import tensorflow as tf

if 'COLAB_TPU_ADDR' in os.environ:
    tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu=tpu_address)
    tf.config.experimental_connect_to_cluster(resolver)
    tf.tpu.experimental.initialize_tpu_system(resolver)
    print("TPU initialized.")
    print("TPU type: ", resolver.cluster_spec().as_dict())
else:
    print("TPU not found. Please ensure that TPU is enabled in your runtime settings.")



In [ ]:
# resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
#
# tf.config.experimental_connect_to_cluster(resolver)
# tf.tpu.experimental.initialize_tpu_system(resolver)
# print("All devices: ", tf.config.list_logical_devices('TPU'))

In [ ]:
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')

In [ ]:
X = list(X)
X_val = list(X_val)
X_test_set = list(X_test_set)

In [ ]:
len(X_val)

In [ ]:
len(Y_vali)

In [ ]:
model_train_x, model_val_x, Y_train, Y_val = train_test_split(X, Y, test_size=0.05, random_state=44)

In [ ]:
train_encodings = tokenizer(model_train_x, max_length=100, truncation=True, padding="max_length", return_tensors='tf')
val_encodings = tokenizer(model_val_x, max_length=100, truncation=True, padding="max_length", return_tensors='tf')
test_encodings = tokenizer(X_val, max_length=100, truncation=True, padding="max_length", return_tensors='tf')

In [ ]:
cluster_encodings = tokenizer(X, max_length=100, truncation=True, padding="max_length", return_tensors='tf')

Subtask A

In [ ]:
def Offense_classifier(input_shape):
    """
    Function creating the model's graph.

    Arguments:
    input_shape -- shape of the input,(max_len,)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 100-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (13 million words)

    Returns:
    model -- a model instance in Keras
    """
    model = TFRobertaModel.from_pretrained('roberta-base')
    layer = model.layers[0]

    # Define sentence_indices as the input of the graph, it should be of shape input_shape and dtype 'int32' (as it contains indices).
    inputs = keras.Input(shape=input_shape, dtype='int32')
    input_masks = keras.Input(shape=input_shape, dtype='int32')

    embeddings = layer([inputs, input_masks])[1]

    X = BatchNormalization(momentum=0.99, epsilon=0.001, center=True, scale=True)(embeddings)

    # Add dropout with a probability of 0.1
    X = Dropout(0.1)(X)

    X = Dense(128,activation='elu',kernel_regularizer=keras.regularizers.l2(0.001))(X)

    X = Dense(32,activation='elu',kernel_regularizer=keras.regularizers.l2(0.001))(X)

    X = Dense(3,activation='elu',kernel_regularizer=keras.regularizers.l2(0.01))(X)

    X = Dense(32,activation='elu',kernel_regularizer=keras.regularizers.l2(0.001))(X)

    X = BatchNormalization(momentum=0.99, epsilon=0.001, center=True, scale=True)(X)

    X = Dense(128,activation='elu',kernel_regularizer=keras.regularizers.l2(0.001))(X)

    X = Dense(1,activation='linear',kernel_regularizer=keras.regularizers.l2(0.01))(X)

    # Add a sigmoid activation
    X = Activation('sigmoid')(X)

    # Create Model instance which converts sentence_indices into X.
    model = keras.Model(inputs=[inputs,input_masks], outputs=[X])


    return model

In [ ]:
strategy = tf.distribute.MirroredStrategy()

In [ ]:
# strategy = tf.distribute.TPUStrategy(resolver)

In [ ]:
class EvaluationMetric(keras.callbacks.Callback):

    def __init__(self, trial_encodings, trial_masks, Y_vali):
        super(EvaluationMetric, self).__init__()
        self.trial_encodings = trial_encodings
        self.trial_masks = trial_masks
        self.Y_vali = Y_vali

    def on_epoch_begin(self, epoch, logs={}):
        print("\nTraining...")

    def on_epoch_end(self, epoch, logs={}):
        print("\nEvaluating...")
        trial_prediction = self.model.predict([self.trial_encodings,self.trial_masks])

        pred = []
        for i in range(0,len(self.Y_vali)):
            num = trial_prediction[i]
            if(num > 0.5):
              num = 1
            else:
              num = 0
            pred.append(num)

        from sklearn.metrics import classification_report
        print(classification_report(Y_vali, pred, digits=3))

evaluation_metric = EvaluationMetric(test_encodings["input_ids"], test_encodings["attention_mask"], Y_vali)

In [ ]:
import tensorflow as tf
print(tf.__version__)


In [ ]:
from transformers import TFRobertaModel

roberta_model = TFRobertaModel.from_pretrained("roberta-base", from_pt=True)
# "from_pt=True" converts a PyTorch model to TensorFlow.


In [ ]:
with strategy.scope():
    model = Offense_classifier((100,))
    optimizer = keras.optimizers.Adam(learning_rate=2e-5)
    loss_fun = [
          tf.keras.losses.BinaryCrossentropy()
    ]
    metric = ['acc']
    model.compile(optimizer=optimizer, loss=loss_fun, metrics=metric)

In [ ]:
model.summary()

In [ ]:
neg, pos = np.bincount(Y)
total = neg + pos
print('Examples:\n    Total: {}\n    Positive: {} ({:.2f}% of total)\n'.format(
    total, pos, 100 * pos / total))

In [ ]:
class_weight = {}
maxi = max(neg, pos)
weight_for_0 = (maxi / (maxi + neg))
weight_for_1 = (maxi / (maxi + pos))

class_weight = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

In [ ]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(filepath='/content/drive/MyDrive/Colab Notebooks/olid_roberta.006 (1).h5',
                                 monitor='val_acc',
                                 verbose=1,
                                 save_weights_only=True,
                                 period=1)

In [ ]:
len(train_encodings)

In [ ]:
len(model_val_x)

In [ ]:
len(Y_val)

In [ ]:
# val 0.05
history = model.fit(
    x = [train_encodings["input_ids"], train_encodings["attention_mask"]],
    y = Y_train,
    validation_data = ([val_encodings["input_ids"],val_encodings["attention_mask"]],Y_val),
    callbacks = [evaluation_metric, checkpoint],
    batch_size = 64,
    shuffle=True,
    epochs=6,
    class_weight = class_weight
)

In [ ]:
history = history
import matplotlib.pyplot as plt
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='lower right')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='lower right')
plt.show()

In [ ]:
test_set_encodings = tokenizer(X_test_set, max_length=100, truncation=True, padding="max_length", return_tensors='tf')

In [ ]:
answer = model.predict([test_set_encodings["input_ids"], test_set_encodings["attention_mask"]])

In [ ]:
pred = []
for i in range(0,len(X_test_set)):

    num = answer[i]
    if(num >= 0.5):
      num = 1
    else:
      num = 0
    pred.append(num)

In [ ]:
con_mat = tf.math.confusion_matrix(labels=Y_test_set, predictions=pred)
print(con_mat)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
figure = plt.figure(figsize=(8, 8))
sns.set(font_scale=1.75)
sns.heatmap(con_mat, annot=True,cmap=plt.cm.viridis,fmt='d', xticklabels=["Sexist","Not Sexist"], yticklabels=["Sexist","Not Sexist"],annot_kws={"size": 15})
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
from sklearn.metrics import f1_score, classification_report

In [ ]:
f1_score(Y_test_set, pred, average='macro')

In [ ]:
print(classification_report(Y_test_set, pred, target_names=["sexist", "not sexist"], digits=3))

In [ ]:
answer_train = model.predict([cluster_encodings["input_ids"], cluster_encodings["attention_mask"]])

In [ ]:
answer_train

Train set analysis

In [ ]:
pred = []
sample = original
count = 0
for i in range(0,len(Y)):

    num = answer_train[i]         # num
    lol = num                     # lol = 0.8
    if(num > 0.5):
      num = 1
    else:
      num = 0
    pred.append(num)
    if(num != Y[i] and Y[i] == 0 and lol >=0.8):
        print(answer_train[i])
        print("Original label: ",labels[i])
        print("Without pre-processing: ",sample[i])
        print("With pre-processing: ",X[i])
        lol = ""
        count += 1

        if(num == 0):
            lol = "Sexist"
        if(num == 1):
            lol = "Not Sexist"
        print("Predicted: " + lol)
        print()

print(count)

Training explanation

In [ ]:
import plotly
import plotly.graph_objs as go
import plotly.express as px

In [ ]:
# 3 neuron output
model.layers[-6].name

In [ ]:
cluster_dense_3 = keras.Model(inputs=model.input, outputs=model.layers[-6].output)
with strategy.scope():
    cluster_3 = cluster_dense_3.predict([cluster_encodings["input_ids"], cluster_encodings["attention_mask"]])

In [ ]:
pred_train = []
temp = 0

for i in range(0,len(Y)):

    num = answer_train[i]

    if(num >= 0.5):
      num = 1
    else:
      num = 0
    pred_train.append(num)

print(temp)

In [ ]:
#

flag = []
count = 0

x_ = []
y_ = []
z_ = []


for i in range(0,len(Y)):
    count = count + 1
    x_.append(cluster_3[i][1])
    y_.append(cluster_3[i][0])
    z_.append(cluster_3[i][2])

    if( answer_train[i] > 0.3 and answer_train[i] < 0.7 ):
        flag.append(2)

    if( answer_train[i] > 0 and answer_train[i] <= 0.3 ):
        flag.append(0)

    if( answer_train[i] >= 0.7 and answer_train[i] < 1 ):
        flag.append(1)

In [ ]:
Counter(flag)

In [ ]:
con_mat = tf.math.confusion_matrix(labels=Y, predictions=pred_train)
print(con_mat)

In [ ]:
pred_colour = []
for i in range(0,len(flag)):
    if flag[i] == 2:
      pred_colour.append("Neutral")
    if flag[i] == 1:
      pred_colour.append("Not sexist")
    if flag[i] == 0:
      pred_colour.append("Sexist")

test_df = pd.DataFrame({'x':x_, 'y':y_, 'z':z_, 'Labels':pred_colour})

fig = px.scatter_3d(test_df, x='x', y='y', z='z', color='Labels')
fig.update_traces(
    marker={
        'size': 1,
        'opacity': 0.7,
        'colorscale' : 'Oryel',
    }
)
fig.update_layout(legend= {'itemsizing': 'constant', 'font_size':18}, font_size=15, scene_aspectmode='cube')
fig.update_layout(width = 750, height = 500)
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))

In [ ]:
pred_colour = []
for i in range(0,len(flag)):
    if pred_train[i] == 1:
      pred_colour.append("Not Sexist")
    if pred_train[i] == 0:
      pred_colour.append("Sexist")

test_df = pd.DataFrame({'X':x_, 'Y':y_, 'Z':z_, 'Labels:':pred_colour})

fig = px.scatter_3d(test_df, x='X', y='Y', z='Z', color='Labels:')
fig.update_traces(
    marker={
        'size': 1,
        'opacity': 1,
        'colorscale' : 'rainbow',
    }
)
fig.update_layout(legend= {'itemsizing': 'constant'}, font_size=14, scene_aspectmode='cube')
fig.update_layout(width = 850, height = 750)
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))

Traning examination end

CLUSTERING
RoBERTa PLM layer

In [ ]:
model.layers[-8].name

In [ ]:
cluster_bert = keras.Model(inputs=model.input, outputs=model.layers[-8].output)
with strategy.scope():
    cl_bert = cluster_bert.predict([cluster_encodings["input_ids"], cluster_encodings["attention_mask"]])

In [ ]:
len(cl_bert)

In [ ]:
flag_bert = []
count = 0

x_bert = []
y_bert = []
z_bert = []


for i in range(0,len(Y)):
    count = count + 1
    x_bert.append(cl_bert[i][0])
    y_bert.append(cl_bert[i][1])
    z_bert.append(cl_bert[i][2])

    if( answer_train[i] > 0.28 and answer_train[i] < 0.8 ):
        flag_bert.append(2)

    if( answer_train[i] > 0 and answer_train[i] <= 0.28 ):
        flag_bert.append(0)

    if( answer_train[i] >= 0.8 and answer_train[i] < 1 ):
        flag_bert.append(1)

print(count)

k-means PLM layer

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
kmeans_bert = KMeans(n_clusters=3, random_state=44).fit(cl_bert)
y_kmeans_bert = kmeans_bert.predict(cl_bert)

In [ ]:
Counter(y_kmeans_bert)

In [ ]:
Counter(flag_bert)

In [ ]:
# 1 index values are sexist
# 0 index values are not sexist
# 2 index values are neutral

count = 0
for i in range(0,len(flag_bert)):
      if flag_bert[i] == 2 and y_kmeans_bert[i] == 1:
        count = count + 1
print(count)

In [ ]:
for i in range(0,len(flag_bert)):
    if(y_kmeans_bert[i] == 0):
      y_kmeans_bert[i] = 1
    elif(y_kmeans_bert[i] == 1):
      y_kmeans_bert[i] = 0
    else:
      y_kmeans_bert[i] = 2

In [ ]:
flag_bert = []
count = 0

x_bert = []
y_bert = []
z_bert = []


for i in range(0,len(Y)):
    count = count + 1
    x_bert.append(cl_bert[i][0])
    y_bert.append(cl_bert[i][1])
    z_bert.append(cl_bert[i][2])

    if( answer_train[i] > 0.3 and answer_train[i] < 0.7 ):
        flag_bert.append(2)

    if( answer_train[i] > 0 and answer_train[i] <= 0.3 ):
        flag_bert.append(0)

    if( answer_train[i] >= 0.7 and answer_train[i] < 1 ):
        flag_bert.append(1)


print(count)

In [ ]:
Counter(flag_bert)

In [ ]:
con_mat = tf.math.confusion_matrix(labels=flag_bert, predictions=y_kmeans_bert)
print(con_mat)

In [ ]:
import sklearn
print(sklearn.metrics.classification_report(flag_bert, y_kmeans_bert, output_dict=False, digits=3))

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine

In [ ]:
centers_bert = kmeans_bert.cluster_centers_

Cosine Similarity Calculation

In [ ]:
svns_off = []
for i in range(0,len(Y)):
    off = cosine(cl_bert[i], centers_bert[1])/2
    svns_off.append(1-off)
print(len(svns_off))

In [ ]:
svns_noff = []
for i in range(0,len(Y)):
    noff = cosine(cl_bert[i], centers_bert[0])/2
    svns_noff.append(1-noff)
print(len(svns_noff))

In [ ]:
svns_neu = []
for i in range(0,len(Y)):
    neu = cosine(cl_bert[i], centers_bert[2])/2
    svns_neu.append(1-neu)
print(len(svns_neu))

k-means PLM plot

In [ ]:
import plotly
import plotly.graph_objs as go
import plotly.express as px

In [ ]:
pred_colour = []
for i in range(0,len(Y)):
    if y_kmeans_bert[i] == 2:
      pred_colour.append("Neutral")
    if y_kmeans_bert[i] == 1:
      pred_colour.append("Not Offensive")
    if y_kmeans_bert[i] == 0:
      pred_colour.append("Offensive")

test_df = pd.DataFrame({'SVNS Offensive':svns_off, 'SVNS Not Offensive':svns_noff, 'SVNS Neutral':svns_neu, 'Labels:':pred_colour})

fig = px.scatter_3d(test_df, x='SVNS Offensive', y='SVNS Not Offensive', z='SVNS Neutral', color='Labels:')
fig.update_traces(
    marker={
        'size': 1,
        'opacity': 1,
        'colorscale' : 'viridis',
    },
)
fig.update_layout(legend= {'itemsizing': 'constant', 'font_size':18}, font_size=15, scene_aspectmode='cube')
fig.update_layout(width = 850, height = 750)
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))

In [ ]:
print("Length of svns_off:", len(svns_off))
print("Length of svns_neu:", len(svns_neu))
print("Length of svns_noff:", len(svns_noff))
print(len(Y))

In [ ]:
num=len(Y)

In [ ]:
pred_krobert = []
# Assuming svns_off, svns_neu, and svns_noff are arrays or lists
for i in range(num):
    if(svns_off[i] > svns_neu[i]+svns_noff[i]):
      pred_krobert.append(0)
    elif (svns_noff[i] > svns_neu[i] and svns_noff[i] > svns_off[i]):
      pred_krobert.append(1)
    elif (svns_neu[i]>svns_noff[i] and svns_neu[i]>svns_off[i]):
      pred_krobert.append(1)
    else:
      pred_krobert.append(0)
print(classification_report(Y, pred_krobert, output_dict=False, digits=3))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

# Assuming svns_off, svns_neu, and svns_noff are arrays or lists
X = list(zip(svns_off, svns_neu, svns_noff))

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Initialize Decision Tree Classifier
decision_tree = DecisionTreeClassifier()

# Train the decision tree model
decision_tree.fit(X_train, Y_train)

# Predict using the trained model
pred_krobert = decision_tree.predict(X_test)

# Print classification report
print(classification_report(Y_test, pred_krobert, digits=3))


Test Explanation

In [ ]:
import plotly
import plotly.graph_objs as go
import plotly.express as px

In [ ]:
# 3 neuron output
model.layers[-6].name

In [ ]:
cluster_dense_3 = keras.Model(inputs=model.input, outputs=model.layers[-6].output)
with strategy.scope():
    test_3 = cluster_dense_3.predict([test_set_encodings["input_ids"], test_set_encodings["attention_mask"]])

In [ ]:
pred_train = []
temp = 0

for i in range(0,len(Y_test_set)):

    num = answer[i]

    if(num >= 0.5):
      num = 1
    else:
      num = 0
    pred_train.append(num)

print(temp)

In [ ]:
#

flag = []
count = 0

x_ = []
y_ = []
z_ = []


for i in range(0,len(Y_test_set)):
    count = count + 1
    x_.append(test_3[i][1])
    y_.append(test_3[i][0])
    z_.append(test_3[i][2])

    if( answer[i] > 0.3 and answer[i] < 0.7 ):
        flag.append(2)

    if( answer[i] > 0 and answer[i] <= 0.3 ):
        flag.append(0)

    if( answer[i] >= 0.7 and answer[i] < 1 ):
        flag.append(1)

In [ ]:
Counter(flag)

In [ ]:
con_mat = tf.math.confusion_matrix(labels=Y_test_set, predictions=pred_train)
print(con_mat)

In [ ]:
pred_colour = []
for i in range(0,len(flag)):
    if flag[i] == 2:
      pred_colour.append("Neutral")
    if flag[i] == 1:
      pred_colour.append("Not sexist")
    if flag[i] == 0:
      pred_colour.append("Sexist")

test_df = pd.DataFrame({'x':x_, 'y':y_, 'z':z_, 'Labels':pred_colour})

fig = px.scatter_3d(test_df, x='x', y='y', z='z', color='Labels')
fig.update_traces(
    marker={
        'size': 1,
        'opacity': 0.7,
        'colorscale' : 'Oryel',
    }
)
fig.update_layout(legend= {'itemsizing': 'constant', 'font_size':18}, font_size=15, scene_aspectmode='cube')
fig.update_layout(width = 750, height = 500)
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))

In [ ]:
pred_colour = []
for i in range(0,len(flag)):
    if pred_train[i] == 1:
      pred_colour.append("Not Sexist")
    if pred_train[i] == 0:
      pred_colour.append("Sexist")

test_df = pd.DataFrame({'X':x_, 'Y':y_, 'Z':z_, 'Labels:':pred_colour})

fig = px.scatter_3d(test_df, x='X', y='Y', z='Z', color='Labels:')
fig.update_traces(
    marker={
        'size': 1,
        'opacity': 1,
        'colorscale' : 'rainbow',
    }
)
fig.update_layout(legend= {'itemsizing': 'constant'}, font_size=14, scene_aspectmode='cube')
fig.update_layout(width = 850, height = 750)
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))

Testing Examination End

CLUSTERING RoBERTA PLM Layer

In [ ]:
model.layers[-8].name

In [ ]:
cluster_bert = keras.Model(inputs=model.input, outputs=model.layers[-8].output)
with strategy.scope():
    test_cl_bert = cluster_bert.predict([test_set_encodings["input_ids"], test_set_encodings["attention_mask"]])

In [ ]:
flag_bert = []
count = 0

x_bert = []
y_bert = []
z_bert = []


for i in range(0,len(Y_test_set)):
    count = count + 1
    x_bert.append(test_cl_bert[i][0])
    y_bert.append(test_cl_bert[i][1])
    z_bert.append(test_cl_bert[i][2])

    if( answer[i] > 0.28 and answer[i] < 0.8 ):
        flag_bert.append(2)

    if( answer[i] > 0 and answer[i] <= 0.28 ):
        flag_bert.append(0)

    if( answer[i] >= 0.8 and answer[i] < 1 ):
        flag_bert.append(1)

print(count)


k-means PLM Layer

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
kmeans_bert = KMeans(n_clusters=3, random_state=44).fit(test_cl_bert)
y_kmeans_bert = kmeans_bert.predict(test_cl_bert)

In [ ]:
Counter(y_kmeans_bert)

In [ ]:
Counter(flag_bert)

In [ ]:
# 1 index values are sexist
# 0 index values are not sexist
# 2 index values are neutral

count = 0
for i in range(0,len(flag_bert)):
      if flag_bert[i] == 2 and y_kmeans_bert[i] == 1:
        count = count + 1
print(count)

In [ ]:
for i in range(0,len(flag_bert)):
    if(y_kmeans_bert[i] == 0):
      y_kmeans_bert[i] = 1
    elif(y_kmeans_bert[i] == 1):
      y_kmeans_bert[i] = 0
    else:
      y_kmeans_bert[i] = 2

In [ ]:
flag_bert = []
count = 0

x_bert = []
y_bert = []
z_bert = []


for i in range(0,len(Y_test_set)):
    count = count + 1
    x_bert.append(test_cl_bert[i][0])
    y_bert.append(test_cl_bert[i][1])
    z_bert.append(test_cl_bert[i][2])

    if( answer[i] > 0.3 and answer[i] < 0.7 ):
        flag_bert.append(2)

    if( answer[i] > 0 and answer[i] <= 0.3 ):
        flag_bert.append(0)

    if( answer[i] >= 0.7 and answer[i] < 1 ):
        flag_bert.append(1)


print(count)

In [ ]:
Counter(flag_bert)

In [ ]:
con_mat = tf.math.confusion_matrix(labels=flag_bert, predictions=y_kmeans_bert)
print(con_mat)

In [ ]:
import sklearn
print(sklearn.metrics.classification_report(flag_bert, y_kmeans_bert, output_dict=False, digits=3))

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine

In [ ]:
centers_bert = kmeans_bert.cluster_centers_

In [ ]:
t_svns_off = []
for i in range(0,len(Y_test_set)):
    off = cosine(test_cl_bert[i], centers_bert[1])/2
    t_svns_off.append(1-off)
print(len(t_svns_off))

In [ ]:
t_svns_noff = []
for i in range(0,len(Y_test_set)):
    noff = cosine(test_cl_bert[i], centers_bert[0])/2
    t_svns_noff.append(1-noff)
print(len(t_svns_noff))

In [ ]:
t_svns_neu = []
for i in range(0,len(Y_test_set)):
    neu = cosine(test_cl_bert[i], centers_bert[2])/2
    t_svns_neu.append(1-neu)
print(len(t_svns_neu))

k-means PLM plot

In [ ]:
import plotly
import plotly.graph_objs as go
import plotly.express as px

In [ ]:
pred_colour = []
for i in range(0,len(Y_test_set)):
    if y_kmeans_bert[i] == 2:
      pred_colour.append("Neutral")
    if y_kmeans_bert[i] == 1:
      pred_colour.append("Not Offensive")
    if y_kmeans_bert[i] == 0:
      pred_colour.append("Offensive")

test_df = pd.DataFrame({'SVNS Offensive':t_svns_off, 'SVNS Not Offensive':t_svns_noff, 'SVNS Neutral':t_svns_neu, 'Labels:':pred_colour})

fig = px.scatter_3d(test_df, x='SVNS Offensive', y='SVNS Not Offensive', z='SVNS Neutral', color='Labels:')
fig.update_traces(
    marker={
        'size': 1,
        'opacity': 1,
        'colorscale' : 'viridis',
    },
)
fig.update_layout(legend= {'itemsizing': 'constant', 'font_size':18}, font_size=15, scene_aspectmode='cube')
fig.update_layout(width = 850, height = 750)
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))


In [ ]:
pred_krobert = []
# Assuming svns_off, svns_neu, and svns_noff are arrays or lists
for i in range(len(Y_test_set)):
    if(t_svns_off[i] > t_svns_neu[i]+t_svns_noff[i]):
      pred_krobert.append(0)
    elif (t_svns_noff[i] > t_svns_neu[i] and t_svns_noff[i] > t_svns_off[i]):
      pred_krobert.append(1)
    elif (t_svns_neu[i]>t_svns_noff[i] and t_svns_neu[i]>t_svns_off[i]):
      pred_krobert.append(1)
    else:
      pred_krobert.append(0)
print(classification_report(Y_test_set, pred_krobert, output_dict=False, digits=3))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

X = list(zip(svns_off, svns_neu, svns_noff))
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
pred_krobert = decision_tree.predict(X_test)
print(classification_report(Y_test, pred_krobert, digits=3))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split

# Assuming svns_off, svns_neu, svns_noff, and Y are defined

# Prepare the data
X = list(zip(svns_off, svns_neu, svns_noff))
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Initialize and train the model
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)

# Predict on the training set
pred_train = decision_tree.predict(X_train)

# Predict on the test set
pred_test = decision_tree.predict(X_test)

# Calculate train accuracy
train_accuracy = accuracy_score(Y_train, pred_train)
print(f'Train Accuracy: {train_accuracy:.3f}')

# Calculate test accuracy
test_accuracy = accuracy_score(Y_test, pred_test)
print(f'Test Accuracy: {test_accuracy:.3f}')

# Print classification report for the test set
print(classification_report(Y_test, pred_test, digits=3))


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

X_train = list(zip(svns_off, svns_neu, svns_noff))
X_test = list(zip(t_svns_off, t_svns_neu, t_svns_noff))

Y_train=Y
Y_test=Y_test_set

decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
pred_krobert = decision_tree.predict(X_test)
print(classification_report(Y_test, pred_krobert, digits=3))


Validation Explanation

In [ ]:
answer_val = model.predict([test_encodings["input_ids"], test_encodings["attention_mask"]])

In [ ]:
import plotly
import plotly.graph_objs as go
import plotly.express as px

In [ ]:
# 3 neuron output
model.layers[-6].name

In [ ]:
cluster_dense_3 = keras.Model(inputs=model.input, outputs=model.layers[-6].output)
with strategy.scope():
    val_3 = cluster_dense_3.predict([test_encodings["input_ids"], test_encodings["attention_mask"]])

In [ ]:
pred_train = []
temp = 0

for i in range(0,len(Y_vali)):

    num = answer_val[i]

    if(num >= 0.5):
      num = 1
    else:
      num = 0
    pred_train.append(num)

print(temp)

In [ ]:
flag = []
count = 0

x_ = []
y_ = []
z_ = []


for i in range(0,len(Y_vali)):
    count = count + 1
    x_.append(val_3[i][1])
    y_.append(val_3[i][0])
    z_.append(val_3[i][2])

    if( answer_val[i] > 0.3 and answer_val[i] < 0.7 ):
        flag.append(2)

    if( answer_val[i] > 0 and answer_val[i] <= 0.3 ):
        flag.append(0)

    if( answer_val[i] >= 0.7 and answer_val[i] < 1 ):
        flag.append(1)

In [ ]:
Counter(flag)

In [ ]:
con_mat = tf.math.confusion_matrix(labels=Y_vali, predictions=pred_train)
print(con_mat)

In [ ]:
pred_colour = []
for i in range(0,len(flag)):
    if flag[i] == 2:
      pred_colour.append("Neutral")
    if flag[i] == 1:
      pred_colour.append("Not sexist")
    if flag[i] == 0:
      pred_colour.append("Sexist")

test_df = pd.DataFrame({'x':x_, 'y':y_, 'z':z_, 'Labels':pred_colour})

fig = px.scatter_3d(test_df, x='x', y='y', z='z', color='Labels')
fig.update_traces(
    marker={
        'size': 1,
        'opacity': 0.7,
        'colorscale' : 'Oryel',
    }
)
fig.update_layout(legend= {'itemsizing': 'constant', 'font_size':18}, font_size=15, scene_aspectmode='cube')
fig.update_layout(width = 750, height = 500)
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))

In [ ]:
pred_colour = []
for i in range(0,len(flag)):
    if pred_train[i] == 1:
      pred_colour.append("Not Sexist")
    if pred_train[i] == 0:
      pred_colour.append("Sexist")

test_df = pd.DataFrame({'X':x_, 'Y':y_, 'Z':z_, 'Labels:':pred_colour})

fig = px.scatter_3d(test_df, x='X', y='Y', z='Z', color='Labels:')
fig.update_traces(
    marker={
        'size': 1,
        'opacity': 1,
        'colorscale' : 'rainbow',
    }
)
fig.update_layout(legend= {'itemsizing': 'constant'}, font_size=14, scene_aspectmode='cube')
fig.update_layout(width = 850, height = 750)
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))

Validation Examination End

CLUSTERING RoBERTA PLM Layer

In [ ]:
model.layers[-8].name

In [ ]:
cluster_bert = keras.Model(inputs=model.input, outputs=model.layers[-8].output)
with strategy.scope():
    val_cl_bert = cluster_bert.predict([test_encodings["input_ids"], test_encodings["attention_mask"]])

In [ ]:
len(Y_vali)

In [ ]:
flag_bert = []
count = 0

x_bert = []
y_bert = []
z_bert = []


for i in range(0,len(Y_vali)):
    count = count + 1
    x_bert.append(val_cl_bert[i][0])
    y_bert.append(val_cl_bert[i][1])
    z_bert.append(val_cl_bert[i][2])

    if( answer_val[i] > 0.28 and answer_val[i] < 0.8 ):
        flag_bert.append(2)

    if( answer_val[i] > 0 and answer_val[i] <= 0.28 ):
        flag_bert.append(0)

    if( answer_val[i] >= 0.8 and answer_val[i] < 1 ):
        flag_bert.append(1)

print(count)

K-Means PLM Layer

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
kmeans_bert = KMeans(n_clusters=3, random_state=44).fit(val_cl_bert)
y_kmeans_bert = kmeans_bert.predict(val_cl_bert)

In [ ]:
Counter(y_kmeans_bert)

In [ ]:
Counter(flag_bert)

In [ ]:
len(flag_bert)

In [ ]:
# 1 index values are sexist
# 0 index values are not sexist
# 2 index values are neutral

count = 0
for i in range(0,len(flag_bert)):
      if flag_bert[i] == 2 and y_kmeans_bert[i] == 1:
        count = count + 1
print(count)

In [ ]:
for i in range(0,len(flag_bert)):
    if(y_kmeans_bert[i] == 0):
      y_kmeans_bert[i] = 1
    elif(y_kmeans_bert[i] == 1):
      y_kmeans_bert[i] = 0
    else:
      y_kmeans_bert[i] = 2

In [ ]:
flag_bert = []
count = 0

x_bert = []
y_bert = []
z_bert = []


for i in range(0,len(Y_vali)):
    count = count + 1
    x_bert.append(val_cl_bert[i][0])
    y_bert.append(val_cl_bert[i][1])
    z_bert.append(val_cl_bert[i][2])

    if( answer_val[i] > 0.3 and answer_val[i] < 0.7 ):
        flag_bert.append(2)

    if( answer_val[i] > 0 and answer_val[i] <= 0.3 ):
        flag_bert.append(0)

    if( answer_val[i] >= 0.7 and answer_val[i] < 1 ):
        flag_bert.append(1)


print(count)

In [ ]:
Counter(flag_bert)

In [ ]:
con_mat = tf.math.confusion_matrix(labels=flag_bert, predictions=y_kmeans_bert)
print(con_mat)

In [ ]:
import sklearn
print(sklearn.metrics.classification_report(flag_bert, y_kmeans_bert, output_dict=False, digits=3))

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine

In [ ]:
centers_bert = kmeans_bert.cluster_centers_

In [ ]:
v_svns_off = []
for i in range(0,len(Y_vali)):
    off = cosine(val_cl_bert[i], centers_bert[1])/2
    v_svns_off.append(1-off)
print(len(v_svns_off))

In [ ]:
v_svns_noff = []
for i in range(0,len(Y_vali)):
    noff = cosine(val_cl_bert[i], centers_bert[0])/2
    v_svns_noff.append(1-noff)
print(len(v_svns_noff))

In [ ]:
v_svns_neu = []
for i in range(0,len(Y_vali)):
    neu = cosine(val_cl_bert[i], centers_bert[2])/2
    v_svns_neu.append(1-neu)
print(len(v_svns_neu))

K-Means PLM Plot

In [ ]:
import plotly
import plotly.graph_objs as go
import plotly.express as px

In [ ]:
pred_colour = []
for i in range(0,len(Y_vali)):
    if y_kmeans_bert[i] == 2:
      pred_colour.append("Neutral")
    if y_kmeans_bert[i] == 1:
      pred_colour.append("Not Offensive")
    if y_kmeans_bert[i] == 0:
      pred_colour.append("Offensive")

test_df = pd.DataFrame({'SVNS Offensive':v_svns_off, 'SVNS Not Offensive':v_svns_noff, 'SVNS Neutral':v_svns_neu, 'Labels:':pred_colour})

fig = px.scatter_3d(test_df, x='SVNS Offensive', y='SVNS Not Offensive', z='SVNS Neutral', color='Labels:')
fig.update_traces(
    marker={
        'size': 1,
        'opacity': 1,
        'colorscale' : 'viridis',
    },
)
fig.update_layout(legend= {'itemsizing': 'constant', 'font_size':18}, font_size=15, scene_aspectmode='cube')
fig.update_layout(width = 850, height = 750)
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))

In [ ]:
pred_krobert = []
# Assuming svns_off, svns_neu, and svns_noff are arrays or lists
for i in range(len(Y_vali)):
    if(v_svns_off[i] > v_svns_neu[i]+v_svns_noff[i]):
      pred_krobert.append(0)
    elif (v_svns_noff[i] > v_svns_neu[i] and v_svns_noff[i] > v_svns_off[i]):
      pred_krobert.append(1)
    elif (v_svns_neu[i]>v_svns_noff[i] and v_svns_neu[i]>v_svns_off[i]):
      pred_krobert.append(1)
    else:
      pred_krobert.append(0)
print(classification_report(Y_vali, pred_krobert, output_dict=False, digits=3))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score

# Prepare the data
X = list(zip(svns_off, svns_neu, svns_noff))
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Initialize and train the model
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)

# Predict on the training set
pred_train = decision_tree.predict(X_train)

# Predict on the test set
pred_test = decision_tree.predict(X_test)

# Calculate train accuracy
train_accuracy = accuracy_score(Y_train, pred_train)
print(f'Train Accuracy: {train_accuracy:.3f}')

# Calculate test accuracy
test_accuracy = accuracy_score(Y_test, pred_test)
print(f'Test Accuracy: {test_accuracy:.3f}')

# Print classification report for the test set
print(classification_report(Y_test, pred_test, digits=3))
